In [157]:
import json
from boto3.dynamodb.types import TypeDeserializer

def ddb_deserialize(r, type_deserializer = TypeDeserializer()):
  return type_deserializer.deserialize({"M": r})

In [162]:
import boto3
import pandas

def ddb_query(query):
  client = boto3.client('dynamodb')
  items = []
  resp = client.execute_statement(Statement=query, Limit=1000)
  items.extend(resp['Items'])
  while 'LastEvaluatedKey' in resp:
    resp = client.execute_statement(Statement=query,Limit=1000,NextToken=resp["NextToken"])
    items.extend(resp['Items'])

  items = [ ddb_deserialize(r) for r in items ]
  return items

def query_sessions_with_sections(query, sectionTable="SessionSection-e3tq3jp6jjh4xjjpvydbqj64e4-dev"):
  
  sessions = ddb_query(query)
  sessions_df = pandas.DataFrame(sessions)

  sections_query= f"""SELECT * FROM "{sectionTable}" WHERE sessionID in {list(sessions_df['id'])} """
  sections = ddb_query(sections_query)
  sections_df = pandas.DataFrame(sections)

  sections_df  = (sections_df.groupby(['sessionID'])
        .agg({'start': lambda x: x.tolist(),'end': lambda x: x.tolist(),})
        .reset_index())

  sessions_df = sessions_df.set_index('id').join(sections_df.set_index('sessionID'), rsuffix='-section').explode(['quaternionTimestamp','linearAccerationTimestamp', 'quaternionW',	'quaternionX',	'quaternionY','quaternionZ', 'linearAccerationX', 'linearAccerationZ', 'linearAccerationY',		])
  sessions_df['isInSection'] = None

  def gen_isInSection(row):
    row['isInSection'] = False
    for i in range(len(row['start'])):
      if row['linearAccerationTimestamp'] > row['start'][i] and row['linearAccerationTimestamp'] < row['end'][i]:
        row['isInSection'] = True
      
  sessions_df.apply(gen_isInSection, axis=1)
  return sessions_df


In [228]:

query= """SELECT * FROM "Session-e3tq3jp6jjh4xjjpvydbqj64e4-dev" WHERE CONTAINS("name", '3 rotations')"""

df = query_sessions_with_sections(query)

df


NameError: name 'query_sessions_with_sections' is not defined